In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,12163
2,Huelva,Confirmados PDIA 14 días,892
3,Huelva,Tasa PDIA 14 días,"174,64752331407382"
4,Huelva,Confirmados PDIA 7 días,462
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,9
629,Municipio de Huelva sin especificar,Total Confirmados,158
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  12163.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3473.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1026 personas en los últimos 7 días 

Un positivo PCR cada 526 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,12163.0,462.0,892.0,510743.0,90.456453,174.647523,120.0
Huelva-Costa,6982.0,303.0,599.0,288115.0,105.166340,207.903094,103.0
Huelva (capital),3473.0,140.0,273.0,143663.0,97.450283,190.028052,54.0
Isla Cristina,602.0,60.0,140.0,21264.0,282.167043,658.389767,22.0
Aljaraque,487.0,40.0,44.0,21260.0,188.146754,206.961430,12.0
Sierra de Huelva-Andévalo Central,1394.0,62.0,100.0,67571.0,91.755339,147.992482,11.0
Condado-Campiña,3630.0,88.0,168.0,155057.0,56.753323,108.347253,10.0
Cartaya,464.0,22.0,35.0,19974.0,110.143186,175.227796,6.0
Aracena,306.0,13.0,19.0,8107.0,160.355249,234.365363,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Almendro (El),24.0,4.0,8.0,826.0,484.261501,968.523002,0.0
Cerro de Andévalo (El),60.0,5.0,18.0,2364.0,211.505922,761.421320,0.0
Isla Cristina,602.0,60.0,140.0,21264.0,282.167043,658.389767,22.0
Cumbres Mayores,20.0,1.0,10.0,1755.0,56.980057,569.800570,0.0
Aroche,69.0,14.0,14.0,3073.0,455.580866,455.580866,1.0
Castaño del Robledo,3.0,1.0,1.0,223.0,448.430493,448.430493,0.0
Encinasola,14.0,4.0,5.0,1305.0,306.513410,383.141762,NaN
Corteconcepción,20.0,2.0,2.0,548.0,364.963504,364.963504,0.0
Villalba del Alcor,115.0,11.0,11.0,3338.0,329.538646,329.538646,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,20.0,1.0,10.0,1755.0,56.980057,569.800570,0.0,0.100000
Villablanca,70.0,1.0,7.0,2848.0,35.112360,245.786517,0.0,0.142857
Lucena del Puerto,112.0,1.0,6.0,3371.0,29.664788,177.988727,0.0,0.166667
Palos de la Frontera,273.0,3.0,14.0,11289.0,26.574542,124.014527,1.0,0.214286
Cerro de Andévalo (El),60.0,5.0,18.0,2364.0,211.505922,761.421320,0.0,0.277778
Lepe,1017.0,12.0,39.0,27431.0,43.746127,142.174912,2.0,0.307692
Palma del Condado (La),582.0,9.0,28.0,10761.0,83.635350,260.198866,0.0,0.321429
Villanueva de los Castillejos,67.0,1.0,3.0,2820.0,35.460993,106.382979,0.0,0.333333
Alosno,50.0,2.0,6.0,3923.0,50.981392,152.944175,1.0,0.333333
